***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Jun, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@ntu.edu.tw </p>
### <p> Master Program in Statistics, CGE, National Taiwan University (NTU), Taipei, Taiwan.</p>
---

# Input

In [1]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime

In [2]:
# Input
PATH1='../data/seedinds.txt'
PATH2='../data/bilabels.txt'
PATH3='../data/seedinds_neighborregions.txt'
PATH4='../data/ResNet18_PlantDisease_45K_Spec200_sampling.csv'
PATH5='../data/ResNet18_PlantDisease_45K_Values_sampling.csv'


#===  parameters ========================================================
TRIALS    = 1
timestr   = ''
REG_COLUMN = "Spec200"
RAW_2D_DATA = False

if RAW_2D_DATA: # 2D
    from CNN_Modules import ME_CNN
else: # 1D
    from CNN_Modules_1D import ME_CNN


# (1)PATH1
df1 = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df1.to_numpy().T[0]
NUM_region = len(region)

print('(1)region:', region)
print('NUM_region:', NUM_region)


# (2)PATH2
df2 = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df2.to_numpy()


# (3)PATH4. Have to be here. The following neighboring process needs this information
df4 = pandas.read_csv(PATH4)
print("(3)original spec table:")
display(df4.head())
#all_region_index  = df.iloc[:,REGION_INDEX_LOC].to_numpy().astype(int)
all_region_index  = df4[REG_COLUMN].to_numpy().astype(int)
print("all_region_index =", all_region_index)
print("len(all_region_index) =",len(all_region_index))


# (4)PATH4 reset label
print("(4)original labels")
display(df4['Label'].value_counts().sort_index(ascending=True))
label=df4['Label'].sort_values().unique() # have to sort it to avoid reusing the previous keys
print("original labels and sorted: ",label)
[df4['Label'].replace(to_replace=label[i], value=i, inplace=True) for i in range(len(label))]  #reset labels
print("reset labels:")
display(df4['Label'].value_counts().sort_index(ascending=True))
test_label_answer = df4["Label"].to_numpy()

# (5)PATH5     convert the embedded data into the pickle file
df5 = pandas.read_csv(PATH5)
test_array = df5.to_numpy()

#save
PATH5='../data/embedded_data.pickle'   #replace original PATH 5
with open(PATH5, 'wb') as f:
    pickle.dump([test_array, test_label_answer], f)
print("test_label_answer and test_array",test_label_answer, "\n", test_array)

(1)region: [ 26 182 169 124  29   1  57  76  53  52 183  59  24 164]
NUM_region: 14
(3)original spec table:


,X1,X2,X3,Class,Label,Spec200
0,1.230895,-11.513212,18.781408,Cherry,2,177
1,3.190353,-14.243707,14.359673,Cherry,2,177
2,1.832576,-12.109021,15.785449,Cherry,2,177
3,1.880559,-12.113366,15.720321,Cherry,2,177
4,5.759654,-13.821998,15.943708,Cherry,2,177


all_region_index = [177 177 177 ...  53  29 144]
len(all_region_index) = 13076
(4)original labels


0      147
2      434
3     4375
4     2708
6     1410
7     1172
8      746
12     826
13    1258
Name: Label, dtype: int64

original labels and sorted:  [ 0  2  3  4  6  7  8 12 13]
reset labels:


0     147
1     434
2    4375
3    2708
4    1410
5    1172
6     746
7     826
8    1258
Name: Label, dtype: int64

test_label_answer and test_array [1 1 1 ... 8 8 8] 
 [[-1.7  -0.2  -1.07 ...  0.46  3.43  0.45]
 [-1.58 -2.04 -1.33 ...  0.03  1.79  0.56]
 [-1.55 -0.9  -1.15 ...  0.51  2.4   1.41]
 ...
 [ 1.39 -0.22 -1.5  ...  1.48  1.93  0.72]
 [ 0.34 -1.75 -3.64 ... -0.76  3.19  1.24]
 [ 0.79 -0.65 -1.39 ... -1.37  4.82 -0.03]]


# Get neighbors

In [3]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  70
[[ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [11 12 14 15 17]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]]
neighbor amount:  10


[[3, 4, 5, 6, 8],
 [],
 [],
 [],
 [],
 [11, 12, 14, 15, 17],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [4]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[26, 3, 4, 5, 6, 8],
 [182],
 [169],
 [124],
 [29],
 [1, 11, 12, 14, 15, 17],
 [57],
 [76],
 [53],
 [52],
 [183],
 [59],
 [24],
 [164]]

In [5]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

In [6]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
# 20240319
if RAW_2D_DATA: # 2D
    print("")
else: # 1D
    test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (13076, 1000)
training size: 3601


# CNN original

In [7]:
# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # fill up training array
        
    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
        


    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

n, p1, p2 0 0 0


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)               

[2 8 2 ... 8 4 8]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:00:28.093357
n, p1, p2 1 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_______________

3240/3240 [==============================] - 0s 85us/step - loss: 0.0545 - accuracy: 0.9864 - val_loss: 0.0320 - val_accuracy: 0.9917
Epoch 6/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0316 - accuracy: 0.9926 - val_loss: 0.0179 - val_accuracy: 0.9972
Epoch 7/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0232 - accuracy: 0.9941 - val_loss: 0.0171 - val_accuracy: 0.9945
Epoch 8/80
3240/3240 [==============================] - 0s 97us/step - loss: 0.0141 - accuracy: 0.9972 - val_loss: 0.0140 - val_accuracy: 0.9972
Epoch 9/80
3240/3240 [==============================] - 0s 97us/step - loss: 0.0122 - accuracy: 0.9985 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 10/80
3240/3240 [==============================] - 0s 100us/step - loss: 0.0129 - accuracy: 0.9969 - val_loss: 0.0115 - val_accuracy: 0.9972
Epoch 11/80
3240/3240 [==============================] - 0s 92us/step - loss: 0.0080 - accuracy: 0.9991 - val_loss: 0.0085 - val_accuracy: 

Epoch 60/80
3240/3240 [==============================] - 0s 87us/step - loss: 9.4865e-05 - accuracy: 1.0000 - val_loss: 9.5493e-04 - val_accuracy: 1.0000
Epoch 00060: early stopping
[[2.5353845e-08 7.1842369e-06 5.7037979e-01 ... 3.3442516e-02
  1.7001394e-02 3.2631124e-05]
 [1.0445978e-08 1.8307921e-09 8.1248283e-02 ... 6.8485910e-01
  3.9726208e-05 1.7365569e-04]
 [1.5013809e-08 1.3081966e-07 1.8059623e-01 ... 2.3023076e-03
  3.2601673e-02 1.2433605e-05]
 ...
 [5.3465321e-15 2.4093573e-14 2.2968489e-08 ... 7.3987212e-05
  1.0467945e-04 1.1288873e-11]
 [4.9024294e-13 1.9233178e-18 7.4057922e-08 ... 6.4508989e-09
  1.4931278e-07 6.3166203e-07]
 [5.7296935e-16 9.9266830e-16 1.3176458e-18 ... 5.3863372e-09
  9.2696645e-08 3.5049398e-22]]
[ 2 11  8 ...  8  4  8]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:00:59.843399
n, p1, p2 3 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              

3240/3240 [==============================] - 0s 87us/step - loss: 1.9523e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 41/80
3240/3240 [==============================] - 0s 87us/step - loss: 2.0278e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 42/80
3240/3240 [==============================] - 0s 88us/step - loss: 1.7597e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 43/80
3240/3240 [==============================] - 0s 85us/step - loss: 1.6476e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 44/80
3240/3240 [==============================] - 0s 86us/step - loss: 1.6426e-04 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 45/80
3240/3240 [==============================] - 0s 85us/step - loss: 1.4570e-04 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 46/80
3240/3240 [==============================] - 0s 84us/step - loss: 1.5825e-04 - accuracy: 1.0000 - va

Train on 3240 samples, validate on 361 samples
Epoch 1/80
3240/3240 [==============================] - 1s 164us/step - loss: 2.5357 - accuracy: 0.1460 - val_loss: 2.4123 - val_accuracy: 0.1496
Epoch 2/80
3240/3240 [==============================] - 0s 88us/step - loss: 2.3189 - accuracy: 0.2886 - val_loss: 1.8807 - val_accuracy: 0.3850
Epoch 3/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.2331 - accuracy: 0.6608 - val_loss: 0.3895 - val_accuracy: 0.9252
Epoch 4/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.1767 - accuracy: 0.9704 - val_loss: 0.0908 - val_accuracy: 0.9695
Epoch 5/80
3240/3240 [==============================] - 0s 92us/step - loss: 0.0596 - accuracy: 0.9849 - val_loss: 0.0359 - val_accuracy: 0.9917
Epoch 6/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0319 - accuracy: 0.9926 - val_loss: 0.0272 - val_accuracy: 0.9945
Epoch 7/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.022

Epoch 56/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.4055e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 57/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.2164e-04 - accuracy: 1.0000 - val_loss: 8.8081e-04 - val_accuracy: 1.0000
Epoch 58/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.3265e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 59/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.0688e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 60/80
3240/3240 [==============================] - 0s 88us/step - loss: 1.0517e-04 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 61/80
3240/3240 [==============================] - 0s 84us/step - loss: 9.7370e-05 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 62/80
3240/3240 [==============================] - 0s 87us/step - loss: 9.5500e-05 - accur

In [8]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

(5, 13076) (5, 13076, 14)


# CNN combination and removal

In [9]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

91

n, p1, p2 0 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84

3240/3240 [==============================] - 0s 87us/step - loss: 2.7324e-04 - accuracy: 1.0000 - val_loss: 5.2421e-04 - val_accuracy: 1.0000
Epoch 47/80
3240/3240 [==============================] - 0s 91us/step - loss: 2.4787e-04 - accuracy: 1.0000 - val_loss: 3.7609e-04 - val_accuracy: 1.0000
Epoch 48/80
3240/3240 [==============================] - 0s 83us/step - loss: 2.4320e-04 - accuracy: 1.0000 - val_loss: 4.1026e-04 - val_accuracy: 1.0000
Epoch 49/80
3240/3240 [==============================] - 0s 86us/step - loss: 2.3166e-04 - accuracy: 1.0000 - val_loss: 3.4950e-04 - val_accuracy: 1.0000
Epoch 50/80
3240/3240 [==============================] - 0s 91us/step - loss: 2.1683e-04 - accuracy: 1.0000 - val_loss: 3.8835e-04 - val_accuracy: 1.0000
Epoch 51/80
3240/3240 [==============================] - 0s 87us/step - loss: 2.1453e-04 - accuracy: 1.0000 - val_loss: 3.4612e-04 - val_accuracy: 1.0000
Epoch 52/80
3240/3240 [==============================] - 0s 82us/step - loss: 2.0342e-04

3240/3240 [==============================] - 0s 91us/step - loss: 0.0496 - accuracy: 0.9917 - val_loss: 0.0302 - val_accuracy: 0.9972
Epoch 6/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0241 - accuracy: 0.9969 - val_loss: 0.0187 - val_accuracy: 1.0000
Epoch 7/80
3240/3240 [==============================] - 0s 83us/step - loss: 0.0180 - accuracy: 0.9975 - val_loss: 0.0155 - val_accuracy: 0.9972
Epoch 8/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.0150 - accuracy: 0.9981 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 9/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0126 - accuracy: 0.9975 - val_loss: 0.0109 - val_accuracy: 0.9972
Epoch 10/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0086 - accuracy: 0.9994 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 11/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0073 - accuracy: 0.9988 - val_loss: 0.0057 - val_accuracy: 1

3240/3240 [==============================] - 0s 87us/step - loss: 0.0055 - accuracy: 0.9994 - val_loss: 0.0106 - val_accuracy: 0.9972
Epoch 15/80
3240/3240 [==============================] - 0s 88us/step - loss: 0.0042 - accuracy: 0.9997 - val_loss: 0.0117 - val_accuracy: 0.9972
Epoch 16/80
3240/3240 [==============================] - 0s 95us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 0.9972
Epoch 17/80
3240/3240 [==============================] - 0s 84us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 18/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 0.9972
Epoch 19/80
3240/3240 [==============================] - 0s 88us/step - loss: 0.0044 - accuracy: 0.9994 - val_loss: 0.0372 - val_accuracy: 0.9861
Epoch 20/80
3240/3240 [==============================] - 0s 89us/step - loss: 0.0205 - accuracy: 0.9951 - val_loss: 0.0082 - val_accurac

...

Epoch 32/80
3081/3081 [==============================] - 0s 89us/step - loss: 6.2261e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 33/80
3081/3081 [==============================] - 0s 90us/step - loss: 5.6817e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 34/80
3081/3081 [==============================] - 0s 89us/step - loss: 4.9266e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 35/80
3081/3081 [==============================] - 0s 89us/step - loss: 4.9168e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 36/80
3081/3081 [==============================] - 0s 91us/step - loss: 4.7495e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 37/80
3081/3081 [==============================] - 0s 94us/step - loss: 4.3485e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 38/80
3081/3081 [==============================] - 0s 89us/step - loss: 4.5251e-04 - accuracy:

2790/2790 [==============================] - 0s 87us/step - loss: 0.0104 - accuracy: 0.9982 - val_loss: 0.0111 - val_accuracy: 0.9968
Epoch 11/80
2790/2790 [==============================] - 0s 88us/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 0.0163 - val_accuracy: 0.9968
Epoch 12/80
2790/2790 [==============================] - 0s 90us/step - loss: 0.0071 - accuracy: 0.9986 - val_loss: 0.0114 - val_accuracy: 0.9968
Epoch 13/80
2790/2790 [==============================] - 0s 86us/step - loss: 0.0064 - accuracy: 0.9986 - val_loss: 0.0104 - val_accuracy: 0.9968
Epoch 14/80
2790/2790 [==============================] - 0s 87us/step - loss: 0.0053 - accuracy: 0.9996 - val_loss: 0.0084 - val_accuracy: 0.9968
Epoch 15/80
2790/2790 [==============================] - 0s 86us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 0.9968
Epoch 16/80
2790/2790 [==============================] - 0s 88us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0061 - val_accurac

3076/3076 [==============================] - 0s 93us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 19/80
3076/3076 [==============================] - 0s 84us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 20/80
3076/3076 [==============================] - 0s 89us/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 21/80
3076/3076 [==============================] - 0s 87us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 0.9971
Epoch 22/80
3076/3076 [==============================] - 0s 91us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 23/80
3076/3076 [==============================] - 0s 86us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 0.9971
Epoch 24/80
3076/3076 [==============================] - 0s 92us/step - loss: 0.0021 - accuracy: 0.9997 - val_loss: 0.0045 - val_accurac

3052/3052 [==============================] - 0s 91us/step - loss: 0.0514 - accuracy: 0.9915 - val_loss: 0.0339 - val_accuracy: 0.9971
Epoch 6/80
3052/3052 [==============================] - 0s 86us/step - loss: 0.0379 - accuracy: 0.9915 - val_loss: 0.0177 - val_accuracy: 1.0000
Epoch 7/80
3052/3052 [==============================] - 0s 90us/step - loss: 0.0185 - accuracy: 0.9977 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 8/80
3052/3052 [==============================] - 0s 88us/step - loss: 0.0157 - accuracy: 0.9971 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 9/80
3052/3052 [==============================] - 0s 92us/step - loss: 0.0119 - accuracy: 0.9977 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 10/80
3052/3052 [==============================] - 0s 88us/step - loss: 0.0117 - accuracy: 0.9980 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 11/80
3052/3052 [==============================] - 0s 94us/step - loss: 0.0093 - accuracy: 0.9984 - val_loss: 0.0099 - val_accuracy: 1

3052/3052 [==============================] - 0s 77us/step - loss: 1.2983e-04 - accuracy: 1.0000 - val_loss: 2.5214e-04 - val_accuracy: 1.0000
Epoch 60/80
3052/3052 [==============================] - 0s 83us/step - loss: 1.2985e-04 - accuracy: 1.0000 - val_loss: 2.6362e-04 - val_accuracy: 1.0000
Epoch 61/80
3052/3052 [==============================] - 0s 88us/step - loss: 1.2835e-04 - accuracy: 1.0000 - val_loss: 2.3027e-04 - val_accuracy: 1.0000
Epoch 62/80
3052/3052 [==============================] - 0s 85us/step - loss: 1.1798e-04 - accuracy: 1.0000 - val_loss: 2.3259e-04 - val_accuracy: 1.0000
Epoch 63/80
3052/3052 [==============================] - 0s 91us/step - loss: 1.4337e-04 - accuracy: 1.0000 - val_loss: 2.4029e-04 - val_accuracy: 1.0000
Epoch 64/80
3052/3052 [==============================] - 0s 88us/step - loss: 1.0896e-04 - accuracy: 1.0000 - val_loss: 2.2135e-04 - val_accuracy: 1.0000
Epoch 65/80
3052/3052 [==============================] - 0s 86us/step - loss: 1.1992e-04

In [10]:
# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
            
    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)


n= 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)          

3154/3154 [==============================] - 0s 90us/step - loss: 2.1968e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 47/80
3154/3154 [==============================] - 0s 84us/step - loss: 2.2549e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 48/80
3154/3154 [==============================] - 0s 88us/step - loss: 2.1414e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 49/80
3154/3154 [==============================] - 0s 86us/step - loss: 2.0226e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 50/80
3154/3154 [==============================] - 0s 88us/step - loss: 1.9461e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 51/80
3154/3154 [==============================] - 0s 95us/step - loss: 1.7863e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 52/80
3154/3154 [==============================] - 0s 87us/step - loss: 1.6912e-04 - accuracy: 1.0000 - va

Epoch 20/80
2983/2983 [==============================] - 0s 89us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 0.9970
Epoch 21/80
2983/2983 [==============================] - 0s 88us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 22/80
2983/2983 [==============================] - 0s 83us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 23/80
2983/2983 [==============================] - 0s 93us/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0060 - val_accuracy: 0.9970
Epoch 24/80
2983/2983 [==============================] - 0s 86us/step - loss: 9.6306e-04 - accuracy: 1.0000 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoch 25/80
2983/2983 [==============================] - 0s 91us/step - loss: 7.7874e-04 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 0.9970
Epoch 26/80
2983/2983 [==============================] - 0s 86us/step - loss: 7.9704e-04 - accuracy: 1.0000 - val_lo

2983/2983 [==============================] - 0s 88us/step - loss: 7.2841e-05 - accuracy: 1.0000 - val_loss: 6.7336e-04 - val_accuracy: 1.0000
Epoch 75/80
2983/2983 [==============================] - 0s 90us/step - loss: 7.1634e-05 - accuracy: 1.0000 - val_loss: 6.6256e-04 - val_accuracy: 1.0000
Epoch 76/80
2983/2983 [==============================] - 0s 90us/step - loss: 6.9962e-05 - accuracy: 1.0000 - val_loss: 5.9725e-04 - val_accuracy: 1.0000
Epoch 77/80
2983/2983 [==============================] - 0s 88us/step - loss: 6.7968e-05 - accuracy: 1.0000 - val_loss: 5.5747e-04 - val_accuracy: 1.0000
Epoch 78/80
2983/2983 [==============================] - 0s 89us/step - loss: 6.5140e-05 - accuracy: 1.0000 - val_loss: 7.2691e-04 - val_accuracy: 1.0000
Epoch 79/80
2983/2983 [==============================] - 0s 87us/step - loss: 6.5603e-05 - accuracy: 1.0000 - val_loss: 5.9141e-04 - val_accuracy: 1.0000
Epoch 80/80
2983/2983 [==============================] - 0s 91us/step - loss: 6.2001e-05

Epoch 35/80
3091/3091 [==============================] - 0s 93us/step - loss: 2.8136e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 36/80
3091/3091 [==============================] - 0s 89us/step - loss: 2.6902e-04 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 37/80
3091/3091 [==============================] - 0s 91us/step - loss: 2.6405e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 38/80
3091/3091 [==============================] - 0s 86us/step - loss: 2.7118e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 39/80
3091/3091 [==============================] - 0s 90us/step - loss: 3.6382e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 40/80
3091/3091 [==============================] - 0s 91us/step - loss: 2.1994e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 41/80
3091/3091 [==============================] - 0s 91us/step - loss: 2.0208e-04 - accuracy:

...

2790/2790 [==============================] - 0s 89us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 22/80
2790/2790 [==============================] - 0s 89us/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 23/80
2790/2790 [==============================] - 0s 89us/step - loss: 8.6123e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 24/80
2790/2790 [==============================] - 0s 87us/step - loss: 8.6721e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 25/80
2790/2790 [==============================] - 0s 85us/step - loss: 9.2305e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 26/80
2790/2790 [==============================] - 0s 92us/step - loss: 6.7660e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 27/80
2790/2790 [==============================] - 0s 81us/step - loss: 6.6159e-04 - accuracy: 1.0000 - val_loss: 

Epoch 75/80
2790/2790 [==============================] - 0s 87us/step - loss: 6.8865e-05 - accuracy: 1.0000 - val_loss: 4.6437e-04 - val_accuracy: 1.0000
Epoch 76/80
2790/2790 [==============================] - 0s 88us/step - loss: 6.7356e-05 - accuracy: 1.0000 - val_loss: 4.4740e-04 - val_accuracy: 1.0000
Epoch 77/80
2790/2790 [==============================] - 0s 88us/step - loss: 6.6228e-05 - accuracy: 1.0000 - val_loss: 4.5703e-04 - val_accuracy: 1.0000
Epoch 78/80
2790/2790 [==============================] - 0s 89us/step - loss: 6.2844e-05 - accuracy: 1.0000 - val_loss: 4.1386e-04 - val_accuracy: 1.0000
Epoch 79/80
2790/2790 [==============================] - 0s 86us/step - loss: 6.0852e-05 - accuracy: 1.0000 - val_loss: 4.3073e-04 - val_accuracy: 1.0000
Epoch 80/80
2790/2790 [==============================] - 0s 88us/step - loss: 5.7814e-05 - accuracy: 1.0000 - val_loss: 4.3396e-04 - val_accuracy: 1.0000
[[4.7988968e-10 1.0380013e-03 8.4350979e-01 ... 2.1975156e-05
  4.0969087e-0

3076/3076 [==============================] - 0s 92us/step - loss: 5.2596e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 37/80
3076/3076 [==============================] - 0s 92us/step - loss: 5.1500e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 38/80
3076/3076 [==============================] - 0s 90us/step - loss: 5.8520e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 39/80
3076/3076 [==============================] - 0s 90us/step - loss: 4.8619e-04 - accuracy: 1.0000 - val_loss: 9.1736e-04 - val_accuracy: 1.0000
Epoch 40/80
3076/3076 [==============================] - 0s 93us/step - loss: 4.3839e-04 - accuracy: 1.0000 - val_loss: 8.3800e-04 - val_accuracy: 1.0000
Epoch 41/80
3076/3076 [==============================] - 0s 88us/step - loss: 4.1644e-04 - accuracy: 1.0000 - val_loss: 9.9497e-04 - val_accuracy: 1.0000
Epoch 42/80
3076/3076 [==============================] - 0s 87us/step - loss: 4.4919e-04 - accuracy:

Train on 3052 samples, validate on 340 samples
Epoch 1/80
3052/3052 [==============================] - 1s 169us/step - loss: 2.3924 - accuracy: 0.2431 - val_loss: 2.1691 - val_accuracy: 0.4324
Epoch 2/80
3052/3052 [==============================] - 0s 90us/step - loss: 1.5898 - accuracy: 0.5511 - val_loss: 0.9169 - val_accuracy: 0.7559
Epoch 3/80
3052/3052 [==============================] - 0s 88us/step - loss: 0.3661 - accuracy: 0.9335 - val_loss: 0.1064 - val_accuracy: 0.9853
Epoch 4/80
3052/3052 [==============================] - 0s 87us/step - loss: 0.0532 - accuracy: 0.9921 - val_loss: 0.0386 - val_accuracy: 0.9971
Epoch 5/80
3052/3052 [==============================] - 0s 86us/step - loss: 0.0275 - accuracy: 0.9951 - val_loss: 0.0244 - val_accuracy: 1.0000
Epoch 6/80
3052/3052 [==============================] - 0s 88us/step - loss: 0.0151 - accuracy: 0.9977 - val_loss: 0.0171 - val_accuracy: 1.0000
Epoch 7/80
3052/3052 [==============================] - 0s 93us/step - loss: 0.018

3052/3052 [==============================] - 0s 89us/step - loss: 8.5398e-05 - accuracy: 1.0000 - val_loss: 2.4193e-04 - val_accuracy: 1.0000
Epoch 56/80
3052/3052 [==============================] - 0s 87us/step - loss: 8.2269e-05 - accuracy: 1.0000 - val_loss: 2.5062e-04 - val_accuracy: 1.0000
Epoch 57/80
3052/3052 [==============================] - 0s 88us/step - loss: 7.8981e-05 - accuracy: 1.0000 - val_loss: 2.2355e-04 - val_accuracy: 1.0000
Epoch 58/80
3052/3052 [==============================] - 0s 90us/step - loss: 7.8015e-05 - accuracy: 1.0000 - val_loss: 2.3285e-04 - val_accuracy: 1.0000
Epoch 59/80
3052/3052 [==============================] - 0s 87us/step - loss: 7.8833e-05 - accuracy: 1.0000 - val_loss: 2.1245e-04 - val_accuracy: 1.0000
Epoch 60/80
3052/3052 [==============================] - 0s 89us/step - loss: 7.3962e-05 - accuracy: 1.0000 - val_loss: 2.3247e-04 - val_accuracy: 1.0000
Epoch 61/80
3052/3052 [==============================] - 0s 86us/step - loss: 7.3003e-05

In [11]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

(91, 13076) (91, 13076, 13) (91, 13)
normal size =  (91, 13076, 13)
(14, 13076) (14, 13076, 13) (14, 13)


In [12]:
print("done")

done
